# 영화 추천 시스템

- 추천 시스템 : 특정 사용자에 대하여 다양한 정보를 활용하여 원하는 콘텐츠를 제공하는 것

- ex) 유튜브, 넷플릭스, 쇼핑몰, 광고 등

## 콘텐츠 기반 필터링 (TMDB 5000 영화 데이터)

- 사용자가 특정한 아이템을 선호하는 경우, 그 아이템과 비슷한 아이템을 추천하는 방식


### 0. 데이터 준비

In [17]:
# - 데이터 읽기
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)

(4803, 20)


In [2]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


### 1. EDA, 피처 선정
- 컬럼 정보
    - id : 영화 아이디
    - title : 영화명
    - genres : 영화 장르
    - vote_average : 영화 평균 평점
    - vote_count : 영화 투표수
    - popularity : 영화 인기
    - keywords : 영화 키워드
    - overview : 영화 개요

※ genres, keywords의 자료형을 확인해보자 !

In [18]:
# 사용할 피처 선정
# 데이터 확인 head(n)
use_columns = ['id', 'title', 'genres', 'vote_average','vote_count', 'popularity', 'keywords','overview']
movies_df = movies[use_columns]

In [41]:
# genres의 자료형
movies_df['genres'][:5]

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [42]:
# keywords의 자료형
movies_df['keywords'][:5]

0    [{"id": 1463, "name": "culture clash"}, {"id":...
1    [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2    [{"id": 470, "name": "spy"}, {"id": 818, "name...
3    [{"id": 849, "name": "dc comics"}, {"id": 853,...
4    [{"id": 818, "name": "based on novel"}, {"id":...
Name: keywords, dtype: object

In [20]:
type(movies_df['keywords'][:5][0])
#문자열임

str

In [19]:
import json
# 작은따옴표로 key, value가 구분이 되어있을 경우 json loads 사용 불가
json.loads(movies_df['genres'][0].replace('\"', '\''))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

exec는 Python 프로그래밍 언어에서 내장 함수 중 하나입니다. 이 함수를 사용하면 문자열로 표현된 Python 코드를 실행할 수 있습니다. 즉, 실행 중에 동적으로 Python 코드를 생성하고 실행할 수 있게 해줍니다.

exec 함수의 기본 구문은 다음과 같습니다:

In [21]:
# exec 문자열을 파이썬 코드처럼 해석 (작은따옴표, 큰따옴표 구분하지 않음)
exec("temp = " + movies_df['genres'][0].replace('\"', '\''))

In [22]:
temp

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

genres와 keywords에서 id는 제외

In [28]:
movies_df['genres']

0       []
1       []
2       []
3       []
4       []
        ..
4798    []
4799    []
4800    []
4801    []
4802    []
Name: genres, Length: 4803, dtype: object

In [23]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : eval(x))
movies_df['keywords'] = movies_df['keywords'].apply(eval)

하나의 문자열로 바꾸기 (새로운 feature로 추가)

In [24]:
#문자열을 파이썬 코드처럼 해석(작은 따움표, 큰 따움표 구분하지 않음)
movies_df['genres'] = movies_df['genres'].apply(lambda genres : [genre_dict['name'] for genre_dict in genres])
movies_df['keywords'] = movies_df['keywords'].apply(lambda keywords : [keyword_dict['name'] for keyword_dict in keywords])

각 영화 장르를 나타내는 문자열 리스트를 하나의 문자열로 합치는 작업을 수행합니다.

In [25]:
movies_df['genre_literal'] = movies_df['genres'].apply(lambda x: ''.join(x))

In [33]:
movies_df.head(3).T #이렇게 column새로만들고 난다음에 T해서 몇개만 뽑으면 쉽게 볼 수 있음. 

,0,1,2
id,19995,285,206647
title,Avatar,Pirates of the Caribbean: At World's End,Spectre
genres,[],[],[]
vote_average,7.2,6.9,6.3
vote_count,11800,4500,4466
popularity,150.437577,139.082615,107.376788
keywords,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
overview,"In the 22nd century, a paraplegic Marine is di...","Captain Barbossa, long believed to be dead, ha...",A cryptic message from Bond’s past sends him o...
genre_literal,,,


CountVectorizer는 텍스트 데이터를 벡터화하는 데 사용되는 기법 중 하나로, 특히 자연어 처리(Natural Language Processing, NLP) 분야에서 널리 사용되는 도구입니다. 이를 사용하면 문서 집합(document corpus)의 각 문서를 단어의 출현 빈도(count)를 기반으로 숫자 벡터로 변환할 수 있습니다.

CountVectorizer는 다음과 같은 단계로 동작합니다:

Tokenization: 문서를 단어(또는 토큰)로 분할합니다. 일반적으로 공백이나 구두점 등을 기준으로 문서를 단어로 나누어줍니다.

Counting: 각 문서에 대해 각 단어의 출현 빈도를 계산합니다.

Vectorization: 각 문서를 단어의 빈도를 나타내는 숫자 벡터로 변환합니다. 이렇게 생성된 벡터는 희소(sparse) 벡터가 될 수 있으며, 각 원소는 해당 단어의 출현 빈도를 나타냅니다.

CountVectorizer는 사이킷런(Scikit-learn) 라이브러리에서 제공되며, 다음과 같이 사용할 수 있습니다:

해당 문자열을 DTM으로 벡터화

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0.0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genre_literal'])
print(genre_mat.shape)

#0이 아닌 0.0으로 해줘야만 되는 군...

(4803, 1394)


※ genre_mat의 자료형은 무엇일까?

In [37]:
genre_array = genre_mat.toarray()


In [30]:
# 데이터를 위와 같이 저장하는 이유는 불필요한 공간의 낭비를 막고, 
# 연산의 효율성을 높이기 위해서이다.
# 모든 데이터 수에 대하여 모든 차원의 데이터를 저장하면 (4803, 276), 공간의 낭비가 심하고, 
# 연산할때도 모든 값들을 계산에 포함해야하기 때문에 필요한 정보만 저장하여 활용하는 것이 공간상, 연산상 이득이 크다.
# 여기서 공간이라는 것은 메모리
# 연산상의 이득인 이유는 한번 코사인 유사도 계산식을 통해 생각해보시면 좋을 것 같습니다.
[0, 1]
[1, 1]
[20, 1]
[26, 1]
...
# 9개까지

# 276

Ellipsis

cosine similarity 계산

In [31]:
import numpy as np

def cos_similarity(a, b):
    
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

cos_similarity(np.array((7.0, 12.4, 256.0, 322.17)), np.array((7.0, 12.4, 256.0, 322.17)))

0.9999999999999998

### 2. 유사도 계산을 통한 추천 시스템 구현
matirx의 코사인 유사도를 계산
- 코사인 유사도 : 두 벡터 간의 cosine 각도를 이용하여 구할 수 있는 두 벡터 간 유사도

In [33]:
genre_array = genre_mat.toarray()

In [34]:
genre_array.shape

(4803, 1394)

In [38]:
cos_similarity(genre_array[0, :], genre_array[1, :])

0.0

In [39]:
## 직접 계산
cos_sim_result = []
for i in range(genre_array.shape[0]): # 4803
    temp = []
    for j in range(genre_array.shape[0]): # 4803
        sim = cos_similarity(genre_array[i, :], genre_array[j, :])
        temp.append(sim)

    cos_sim_result.append(temp)


KeyboardInterrupt: 

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


genre_sim에서 특정 영화와 유사도가 높은 순서대로 정렬
- argsort : 값들의 배열에서 데이터를 정렬한 **index** 반환


argsort는 주로 배열 또는 리스트와 같은 시퀀스 데이터를 정렬할 때 사용되는 함수입니다. 이 함수는 정렬된 순서대로 원래 요소의 인덱스를 반환합니다. NumPy 라이브러리에서 주로 사용되며, 파이썬의 기본 리스트에는 내장된 argsort 함수가 없습니다.

간단한 예를 들어 설명하겠습니다. 다음과 같은 리스트가 있다고 가정해봅시다:

python
Copy code
data = [8, 2, 5, 1, 9]
이 리스트를 정렬하기 위해서는 sort() 메서드나 sorted() 함수를 사용할 수 있습니다. 하지만 argsort를 사용하면 원래 요소의 인덱스가 정렬된 순서대로 반환됩니다. NumPy를 사용하는 경우에는 다음과 같이 할 수 있습니다:

python
Copy code
import numpy as np

data = [8, 2, 5, 1, 9]
indices = np.argsort(data)

print(indices)
출력 결과는 다음과 같습니다:

csharp
Copy code
[3 1 2 0 4]
argsort() 함수는 원래 리스트 data의 정렬 순서대로 인덱스를 반환하여 [3, 1, 2, 0, 4]와 같은 결과를 얻게 됩니다. 이는 data[3]이 최소값, data[1]이 두 번째로 작은 값, data[2]가 세 번째로 작은 값, data[0]이 네 번째로 작은 값, 그리고 data[4]가 최대값을 가지도록 정렬됨을 의미합니다.

argsort 함수를 사용하면, 정렬된 순서대로 원래 데이터의 인덱스를 알 수 있어서 유용한 경우가 있습니다. 예를 들어, 원래 데이터를 유지한 채로 정렬된 데이터를 얻거나, 데이터의 순위를 계산하는 등 다양한 용도로 활용될 수 있습니다.







In [40]:
temp = [1, 2, 3, 4, 5]

In [42]:
temp[::-1]#.reverse()

[5, 4, 3, 2, 1]

In [45]:
# 내림차순 정렬을 위해 -1 옵션을 추가로 준다
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0  870   46 ... 3172 3173 2401]]


In [46]:
# 1. 영화 제목으로 index 탐색
movie_idx = movies_df[movies_df['title']=="The Godfather"].index.values
movies_df[movies_df['title']=="The Godfather"].index.values

array([3337], dtype=int64)

In [47]:
recommend_idxs = genre_sim_sorted_ind[movie_idx, :5].reshape(-1)
genre_sim_sorted_ind[movie_idx, :5].reshape(-1)

array([3636,  883, 1149, 3337, 1464], dtype=int64)

추천 영화 DataFrame 반환 함수

In [1]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[['title', 'vote_average']]

NameError: name 'movies_df' is not defined

- Vote Average Feature 활용

적은 수의 사람이 투표한 경우, 평점이 정말 유효한지 판단하는 데에는 어려움이 있을 수 있다.

In [ ]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

영화 선정을 위한 가중치 계산식

- v: 개별 영화에 평점을 투표한 횟수  
- m: 평점을 부여하기 위한 최소 투표 횟수  
- R: 개별 영화에 대한 평균 평점  
- C: 전체 영화에 대한 평균 평점  

최소 투표 횟수를 전체의 60% 지점으로 지정

In [ ]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ((v/(v+m)) * R) + ((m / (v + m)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head(2)

가중치평점을 적용한 추천 영화 DataFrame 반환 함수

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index, : (top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

## 아이템 기반 최근접 이웃 협업 필터링 (MovieLens 리뷰 데이터)

### 0. 데이터 준비

In [3]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./movies.csv')
ratings = pd.read_csv('./ratings.csv')
movies.shape, ratings.shape

((9742, 3), (100836, 4))

In [4]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [ ]:
ratings.head(2)

### 1. EDA, 데이터 정리

- ratings와 movie를 movieId를 기준으로 merge

In [6]:
rating_movies = pd.merge(ratings, movies, on = 'movieId') #데이터베이스에 테이블 join하는 느낌 

In [7]:
rating_movies[:2].T

,0,1
userId,1,5
movieId,1,1
rating,4.0,4.0
timestamp,964982703,847434962
title,Toy Story (1995),Toy Story (1995)
genres,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Animation|Children|Comedy|Fantasy


pivot table를 활용하여, rating에 대하여 userId, title로 이루어진 데이터프레임으로 변환

In [9]:
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')
ratings_matrix.fillna(0, inplace=True)
ratings_matrix.head(2)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


영화에 대한 리뷰 갯수를 벡터로 사용하기 위해 transpose

In [13]:
ratings_matrix_T = ratings_matrix.T

코사인 유사도 측정

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(2)

MemoryError: Unable to allocate 721. MiB for an array with shape (9719, 9719) and data type float64

추천영화 DataFrame을 반환하는 함수

In [10]:
def find_sim_movie_item(df, title_name, top_n=10):
    title_movie_sim = df[[title_name]].drop(title_name, axis=0)

    return title_movie_sim.sort_values(title_name, ascending=False)[:top_n]

In [11]:
find_sim_movie_item(item_sim_df, 'Godfather, The (1972)')

NameError: name 'item_sim_df' is not defined

In [ ]:
find_sim_movie_item(item_sim_df, 'Inception (2010)')